<a href="https://colab.research.google.com/github/RodrigoCTroncoso/Agentes-de-IA-Alura-Google-Gemini/blob/main/Imers%C3%A3o_Agentes_de_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Aula 1


Instalando o necessário para fazer funcionar :

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00


Agora está trazendo userdata (no caso GEMINI_API_KEY) do google.colab
E vincula GOOGLE_API_KEY a chave de acesso que pegamos no google studio.

In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Define alguns parametros para a IA, como modelo que será usado e "temeperatura" (nível de criatividade).

In [10]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
resp_test = llm.invoke("Quantos dias de férias eu tenho na miha empresa?")
print(resp_test.content)

In [13]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."

    """
    decisao: AUTO_RESOLVER
    urgencia: BAIXA
    campos_faltantes: []
    """
)

In [14]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriangemOut (BaseModel):
  decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  ugencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)


In [15]:
llm_triagem = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

In [16]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriangemOut)

def triagem(mensagem: str)-> Dict:
  saida: TriangemOut = triagem_chain.invoke([
      SystemMessage(content = TRIAGEM_PROMPT),
      HumanMessage(content = mensagem)
  ])


  return saida.model_dump()

In [17]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remotos. Como faço?",
          "Posso Reembolsar cursos da Alura?",
          "Quantas capivras tem no Rio Pinheiros?"
          ]

In [18]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta:{triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta:{'decisao': 'AUTO_RESOLVER', 'ugencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remotos. Como faço?
 -> Resposta:{'decisao': 'ABRIR_CHAMADO', 'ugencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso Reembolsar cursos da Alura?
 -> Resposta:{'decisao': 'AUTO_RESOLVER', 'ugencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivras tem no Rio Pinheiros?
 -> Resposta:{'decisao': 'PEDIR_INFO', 'ugencia': 'BAIXA', 'campos_faltantes': []}

